# Import libraries

In [1]:
import pandas as pd
import numpy as np
import math
import os
import sys
import json
import time
sys.path.append("../scripts/")
from ActualADM import *
from LinearizedADM import *
from DeadlockElimination import *

# Current and parent directories

In [2]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Read zone information

In [3]:
zones = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Zone-Info')
num_zones = len(zones)

In [4]:
adm_algo = "DBSCAN"
for house_name in ['A']:
    for occupant_id in ['1', '2']:
        for num_timeslots in range(200, 1401, 200):
            print("Actual: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo) + ", Number of Timeslots: " + str(num_timeslots))                  
            dataframe = pd.read_csv(str(parent_directory) + '/data/cleaned/' + 'Cleaned-Dataframe_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.csv')                                                                                                       
            actual_adm = ActualADM(adm_algo, dataframe, house_name, occupant_id, 1440, num_zones)
            list_time_min, list_time_max = actual_adm.noise_augmented_range_calculation()
            analytics = DeadlockElimination(num_timeslots, num_zones, list_time_min, list_time_max)
            deadlock_info = analytics.deadlock_elimination()            
            output_filename = str(parent_directory) + '/data/time-limited-deadlock-elimination/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '_Timeslots-' + str(num_timeslots) + '.json'

            with open(output_filename, "w") as json_file:
                json.dump(deadlock_info, json_file)

House A Occupant 1
Execution Time: 6.929789066314697
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 200
House A Occupant 1
Execution Time: 22.58213996887207
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 400
House A Occupant 1
Execution Time: 51.36706471443176
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 600
House A Occupant 1
Execution Time: 228.4193720817566
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 800
House A Occupant 1
Execution Time: 159.7570879459381
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 1000
House A Occupant 1
Execution Time: 223.94544792175293
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 1200
House A Occupant 1
Execution Time: 301.26544523239136
Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 1400
House A Occupant 2
Execution Time: 29.171215772628784
Actual: House: A, Occupant: 2, ADM Algo: DBSCAN, Num

In [5]:
adm_algo = "DBSCAN"
for house_name in ['A']:
    for occupant_id in ['1', '2']:
        for num_timeslots in range(200, 1401, 200):
            start_time = time.time()
            print("Actual: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo) + ", Number of Timeslots: " + str(num_timeslots))                  
            
            filename = str(parent_directory) + '/data/time-limited-deadlock-elimination/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '_Timeslots-' + str(num_timeslots) + '.json'

            with open(filename, "r") as file:
                data = json.load(file)

            rewards = [0, 1, 2, 4, 3]
            states = data["States"]
            next_states = data["Next-States"]

            my_graph = nx.DiGraph()         
            for i in range(len(states)):
                state = states[i]
                arrival_time = int(state.split('-')[0])
                arrival_zone = int(state.split('-')[1])
                state_stay_duration = int(state.split('-')[2])
                try:
                    for j in next_states[str(i)]:
                        if j == -1:
                            my_graph.add_edge(state, '0-0-0', weight = 0)
                            continue

                        next_state = states[j]
                        next_arrival_time = int(next_state.split('-')[0])
                        next_arrival_zone = int(next_state.split('-')[1])
                        next_state_stay_duration = int(next_state.split('-')[2])

                        if next_arrival_zone == arrival_zone:
                            my_graph.add_edge(state, next_state, weight = (next_state_stay_duration - state_stay_duration) * rewards[arrival_zone])
                        else:
                            my_graph.add_edge(state, next_state, weight = next_state_stay_duration  * rewards[next_arrival_zone])
                except:
                    pass

            source_node = '0-0-0'

            current_source_nodes = [node for node in my_graph.nodes if my_graph.in_degree(node) == 0]

            # remove nodes whose arrival_time! = 0 and does not have any preceeding node
            nodes_to_remove = []

            for node in current_source_nodes:
                arrival_time = int(node.split('-')[0])
                if arrival_time != 0:
                    nodes_to_remove.append(node)

            current_source_nodes = [node for node in current_source_nodes if node not in nodes_to_remove]

            for node in current_source_nodes:
                node_state_stay_duration = int(node.split('-')[2])
                my_graph.add_edge(source_node, node, weight = node_state_stay_duration)

            # Get all nodes reachable from the source node
            reachable_nodes = nx.descendants(my_graph, source_node)
            reachable_nodes.add(source_node)

            # Create a subgraph containing only the reachable nodes
            my_graph = my_graph.subgraph(reachable_nodes)    

            longest_path = nx.dag_longest_path(my_graph, weight = 'weight')    
            len_longest_path = nx.dag_longest_path_length(my_graph, weight = 'weight')

            attack_schedule = {"Longest-Path" : longest_path, "Longest-Path-Length" : len_longest_path, "Number-of-Nodes" : my_graph.number_of_nodes(), "Number-of-Edges:" : my_graph.number_of_edges()}
            print("Execution time", time.time() - start_time_time_time)            

Actual: House: A, Occupant: 1, ADM Algo: DBSCAN, Number of Timeslots: 200


NameError: name 'nx' is not defined